In [14]:

import io.qdrant.client.ConditionFactory
import io.qdrant.client.ConditionFactory.range
import io.qdrant.client.QdrantClient
import io.qdrant.client.grpc.Points
import org.springframework.beans.factory.annotation.Autowired
import org.springframework.stereotype.Component
import io.qdrant.client.QdrantGrpcClient
import io.qdrant.client.grpc.Points.Condition
import io.qdrant.client.grpc.Points.Filter
import org.springframework.ai.embedding.EmbeddingModel
import org.springframework.ai.transformers.TransformersEmbeddingModel
import org.springframework.ai.chat.model.ChatModel
import org.springframework.core.io.DefaultResourceLoader
import java.io.File





In [12]:
fun embeddingModel(): EmbeddingModel {
    val dataFile = File("./model.onnx_data")
    if (!dataFile.exists()) {
        val resource = DefaultResourceLoader().getResource("classpath:transformer/model.onnx_data")
        if (resource.exists()) {
            resource.inputStream.use { input ->
                dataFile.outputStream().use { output ->
                    input.copyTo(output)
                }
            }
        }
    }
    val embeddingModel = TransformersEmbeddingModel()
    embeddingModel.setModelResource("classpath:transformer/model.onnx")
    embeddingModel.setTokenizerResource("classpath:transformer/tokenizer.json")
    embeddingModel.setTokenizerOptions(mapOf(
        "addSpecialTokens" to "true",
        "truncation" to "true",
        "padding" to "true",
        "stride" to "0"
    ))
    embeddingModel.setModelOutputName("token_embeddings")
    embeddingModel.setDisableCaching(true)
    embeddingModel.afterPropertiesSet()
    return embeddingModel
}
embeddingModel().dimensions()

1024

In [13]:
val collectionName = ""
val vectorname = ""
val sentence = ""

fun query(qdrantClient: QdrantClient): List<Points.ScoredPoint> {
    return qdrantClient.searchAsync(
        Points.SearchPoints.newBuilder()
            .setCollectionName(collectionName)
            .setVectorName(vectorname)
            .addAllVector(embeddingModel().embed(sentences).toList())
            .setFilter(
                Points.Filter.newBuilder()
                    .addShould(
                        Points.Condition.newBuilder()
                            .setField(Points.FieldCondition.newBuilder()
                                .setKey("")
                                .setMatch(
                                    Points.Match.newBuilder()
                                        .setText("")
                                )
                            )
                            .setField(Points.FieldCondition.newBuilder()
                                .setKey("")
                                .setMatch(
                                    Points.Match.newBuilder()
                                        .setText("")
                                )
                            )
                    ).build()
            )
            .setWithVectors(Points.WithVectorsSelector.newBuilder().setEnable(true))
            .setWithPayload(Points.WithPayloadSelector.newBuilder().setEnable(true))
            .setLimit(9)
            .setScoreThreshold(0.3f)
            .build()
    ).get()
}

var client: QdrantClient = QdrantClient(
    QdrantGrpcClient.newBuilder("localhost", 6334, false).build()
)
query(client).joinToString("\n\n") { "id: ${it.id}, content: ${it.payloadMap.toString()}, score: ${it.score}" }

org.jetbrains.kotlinx.jupyter.exceptions.ReplInterruptedException: The execution was interrupted

In [15]:
File("./model.onnx_data").delete()

true